# Wykład 10 - Wzorce Projektowe

*"Wzorzec opisuje problem, który powtarza się wielokrotnie w danym środowisku, oraz podaje istotę jego rozwiązania w taki sposób, aby można było je zastosować miliony razy bez potrzeby powtarzania tej samej pracy"*
**Christopher Alexander "A pattern language", 1977**

![](https://brasil.cel.agh.edu.pl/~09sbfraczek/images/wzorce/design.jpg)

- Kreacyjne (konstrukcyjne, creational design patterns) - opisują elastyczne sposoby tworzenia obiektów uniezależniają system od sposobu tworzenia obekt
    - Metoda Wytwórcza (Factory Method) 
    - **Budowniczy (Builder)**
    - **Fabryka (Factory)**
    - Prototyp (Prototype)
    - **Singleton**

- Strukturalne (structural design patterns) - opisują sposob konstrukcji struktur obiektowych korzystają z dziedziczenia i delegacji
    - **Adapter**
    - Dekorator (Decorator)
    - Fasada (Facade)
    - Kompozyt (Composite)
    - Most (Bridge)
    - Pełnomocnik (Proxy)
    - Pyłek (Flyweight)

- Behawioralne (czynnościowe, behavioral design patterns) - opisują algorytmy i przydział odpowiedzialności charakteryzują sposob interakcji między obiektami
    - Interpreter
    - Metoda Szablonowa (Template Method)
    - Iterator
    - Łańcuch Zobowiązań (Chain of Responsibility)
    - **Mediator**
    - **Obserwator (Observer)**
    - Odwiedziający (Visitor)
    - Pamiątka (Memento)
    - Polecenie (Command)
    - Stan (State)
    - Strategia (Strategy)


## Singleton

- Zapewnia że klasa ma tylko jedną instancję
- Zapewnia globalny dostęp do tej instancji

![](https://refactoring.guru/images/patterns/diagrams/singleton/structure-en-2x.png)

In [30]:
// java
// leniwa inicjalizacja

final public class Singleton {

    private static Singleton instance = null;

    private Singleton() {}
    
    private int counter = 0;

    public static Singleton getInstance() {
        if (instance == null)
            instance = new Singleton();
        return instance;
    }
    
    public int getCounter(){
        return counter;
    }
    
    public void increaseCounter(){
        counter++;
    }
    
    public void resetCounter(){
        counter = 0;
    }
}

In [31]:
Singleton singleton = new Singleton();

CompilationException: 

In [32]:
Singleton singleton = Singleton.getInstance();

In [33]:
singleton.getCounter()

0

In [34]:
Singleton second = Singleton.getInstance();

In [35]:
second.increaseCounter();
singleton.getCounter();

1

In [36]:
second.resetCounter();
singleton.getCounter();

0

In [4]:
// kotlin
val lazyInt: Int by lazy {0} // leniwa inicjalizacja

In [5]:
lazyInt

0

In [6]:
object Singleton { // singleton
    var counter = 0
    
    fun increaseCounter(){counter++}
    fun resetCounter(){counter = 0}
}

In [8]:
Singleton.counter

0

In [9]:
Singleton.increaseCounter()
Singleton.counter

1

W języku Kotlin wzorzec `Singleton` jest używany jako zamiennik dla składowych i pól statycznych (`static`), które nie istnieją w tym języku programowania. `Singleton` tworzy się po prostu deklarując obiekt (`object`). W przeciwieństwie do klasy, obiekt nie może mieć konstruktora, ale bloki `init` są dozwolone, jeśli potrzebny jest kod inicjalizacyjny.

Obiekty towarzyszące (`companion object`) to obiekty `Singleton`, których właściwości i funkcje są powiązane z klasą, ale nie z instancją tej klasy.

## `Double-checked Singleton`

In [ ]:
public class Singleton {

    private static volatile Singleton instance = null;

    private Singleton() {}

    public static Singleton getInstance() {
        if (instance == null) {
            synchronized(Singleton.class) {
                if (instance == null) {
                    instance = new Singleton();
                }
            }
        }

        return instance;
    }
}

## Delegaty

In [35]:
import kotlin.reflect.KProperty

class A {
    var s: String by Delegate()
}

class Delegate {
    operator fun getValue(thisRef: Any?, property: KProperty<*>): String {
        return "$thisRef, thank you for delegating '${property.name}' to me!"
    }

    operator fun setValue(thisRef: Any?, property: KProperty<*>, value: String) {
        println("$value has been assigned to '${property.name}' in $thisRef.")
    }
    
    // KProperty - reprezentuje właściwość (val lub var)
}

In [36]:
val a = A()
println(a.s)

Line_34$A@5b5e42df, thank you for delegating 's' to me!


In [37]:
println(a)

Line_34$A@5b5e42df


In [38]:
a.s = "ZMIANA"

ZMIANA has been assigned to 's' in Line_34$A@5b5e42df.


In [39]:
class B {
    var w: String by Delegate()
}

val b = B()
println(b.w)

Line_38$B@682edc3b, thank you for delegating 'w' to me!


In [40]:
println(b)

Line_38$B@682edc3b


computed!
Hello
Hello


## Przeciążanie operatorów

In [47]:
data class Point(val x: Int, val y: Int)

operator fun Point.unaryMinus() = Point(-x, -y)

val point = Point(10, 20)


println(-point)


Point(x=-10, y=-20)


In [49]:
operator fun Point.inc() = Point(x + 1, y + 1)

Line_48.jupyter-kts (1:1 - 9) 'operator' modifier is inapplicable on this function: receiver must be a supertype of the return type

In [57]:
data class Point2(val x: Int, val y: Int){
    operator fun inc() = Point2(x + 1, y + 1)
    operator fun dec() = Point2(x - 1, y - 1)
}

In [58]:
var point2 = Point2(1, 1)
point2++
println(point2)

Point2(x=2, y=2)


In [59]:
point2--
println(point2)

Point2(x=1, y=1)


In [96]:
operator fun Point.plus(other: Point) = Point(x + other.x, y + other.y)

In [95]:
val p1 = Point(1, 1)
val p2 = Point(2, 9)
val p3 = p1 + p2
println(p3)

Point(x=3, y=10)


## Delegat z obserwatorem

In [69]:
import kotlin.properties.Delegates

class User {
    var name: String by Delegates.observable("<no name>") {
        prop, old, new -> println("$old -> $new")
    }
}

val user = User()
println(user.name)
user.name = "first"
user.name = "second"
user.name = "third"

<no name>
<no name> -> first
first -> second
second -> third


## Delegat wetowalny

In [70]:
var max: Int by Delegates.vetoable(0) { 
    property, oldValue, newValue -> newValue > oldValue
}

println(max)

max = 10
println(max)

max = 5
println(max)

0
10
10


In [80]:
var max: Int by Delegates.vetoable(0) { _, oldValue, newValue ->
    if (newValue > oldValue) true else throw IllegalArgumentException("New value must be larger than old value.")
}

println(max)

max = 10
println(max)

try{
    max = 5
}
catch (e: IllegalArgumentException) {e}

0
10


java.lang.IllegalArgumentException: New value must be larger than old value.

In [84]:
class MyClass {
   var newName: Int = 0
   var oldName: Int by this::newName
}

val myClass = MyClass()
myClass.oldName = 42
println(myClass.newName)


42


## Delegat do konstruktora

In [91]:
data class User(val map: Map<String, Any?>) {
    val name: String by map
    val age: Int     by map
}

val user = User(mapOf(
    "name" to "Rafał Lewandków",
    "age"  to 30
))

user

User(map={name=Rafał Lewandków, age=30})

In [87]:
user.name

Rafał Lewandków

In [88]:
user.age

30

## Method chaining

In [6]:
class Person {
    private String name;
    private int age;
    
    public Person setName(String name)
    {
        this.name = name;
        return this;
    }
    
    public Person setAge(int age)
    {
        this.age = age;
        return this;
    }
    
    void display()
    {
        System.out.println("Name = " + name + "; wiek = " + age);
    }
}

In [9]:
new Person()
    .setName("Rafal")
    .setAge(20)
    .display();

Name = Rafal; wiek = 20


In [1]:
class Person {
    private var name: String = ""
    private var age: Int = 0

    fun setName(name: String): Person {
        this.name = name
        return this
    }

    fun setAge(age: Int): Person {
        this.age = age
        return this
    }

    fun display() {
        println("Name = $name; wiek = $age")
    }
}

In [2]:
Person()
    .setName("Rafal")
    .setAge(25)
    .display();

Name = Rafal; wiek = 25


## Budowniczy

Wzorzec projektowy Budowniczy (Builder) jest wzorcem projektowym, który pozwala na tworzenie skomplikowanych obiektów, krok po kroku.

Jego celem jest rozdzielenie procesu tworzenia obiektu od jego reprezentacji, dzięki czemu różne sposoby tworzenia tego samego typu obiektów mogą być zdefiniowane w różnych klasach budowniczych.

Budowniczy składa się z dwóch głównych składników:

- Klasa Product - reprezentuje obiekt, który jest tworzony. Może składać się z różnych składników, które są ustawiane przez budowniczego.
- Klasa Builder - jest interfejsem lub klasą abstrakcyjną, która definiuje metody pozwalające na ustawienie poszczególnych składników obiektu.

Korzystając z wzorca Budowniczy, można uniknąć tworzenia skomplikowanych konstruktorów z dużą liczbą argumentów, a także uniknąć problemów związanych z nieprawidłowymi stanami obiektów, które mogą powstać przy tworzeniu obiektów bezpośrednio przy użyciu konstruktora.

![](https://refactoring.guru/images/patterns/diagrams/builder/problem2-2x.png?id=5e7975a91c0e4f4ba960f908cc9c2ea2)

![](https://refactoring.guru/images/patterns/diagrams/builder/solution1-2x.png?id=a9c2ab02f0b2aca1a7512022194dd113)

![](https://refactoring.guru/images/patterns/diagrams/builder/structure-2x.png?id=dca1b1508e23c266cbedc80ffb84311a)

In [1]:
class Computer {
    var CPU: String = ""
    var RAM: String = ""
    var storage: String = ""
    var GPU: String = ""

    override fun toString(): String {
        return "Computer(CPU='$CPU', RAM='$RAM', storage='$storage', GPU='$GPU')"
    }
}

In [ ]:
abstract class ComputerBuilder {
    protected var computer = Computer()

    abstract fun buildCPU()
    abstract fun buildRAM()
    abstract fun buildStorage()
    abstract fun buildGPU()

    fun getComputer(): Computer {
        return computer
    }
}

In [ ]:
class GamingComputerBuilder : ComputerBuilder() {
    override fun buildCPU() {
        computer.CPU = "i7 9700K"
    }

    override fun buildRAM() {
        computer.RAM = "32GB DDR4"
    }

    override fun buildStorage() {
        computer.storage = "1TB SSD + 2TB HDD"
    }

    override fun buildGPU() {
        computer.GPU = "RTX 3080"
    }
}

In [ ]:
class OfficeComputerBuilder : ComputerBuilder() {
    override fun buildCPU() {
        computer.CPU = "i5 8400"
    }

    override fun buildRAM() {
        computer.RAM = "16GB DDR4"
    }

    override fun buildStorage() {
        computer.storage = "500GB SSD"
    }

    override fun buildGPU() {
        computer.GPU = "N/A"
    }
}

In [ ]:
class ComputerDirector {
    private var computerBuilder: ComputerBuilder = GamingComputerBuilder()

    fun setComputerBuilder(computerBuilder: ComputerBuilder) {
        this.computerBuilder = computerBuilder
    }

    fun buildComputer() {
        computerBuilder.buildCPU()
        computerBuilder.buildRAM()
        computerBuilder.buildStorage()
        computerBuilder.buildGPU()
    }

    fun getComputer(): Computer {
        return computerBuilder.getComputer()
    }
}

In [ ]:
val computerDirector = ComputerDirector()

In [ ]:
computerDirector.setComputerBuilder(GamingComputerBuilder())
computerDirector.buildComputer()
val gamingComputer = computerDirector.getComputer()

In [ ]:
computerDirector.setComputerBuilder(OfficeComputerBuilder())
computerDirector.buildComputer()
val officeComputer = computerDirector.getComputer()

`Director` wzoru projektowego Budowniczy odpowiada za tworzenie konkretnych instancji produktów poprzez składanie ich z pól składowych przy użyciu konkretnego budowniczego. `Director` zarządza procesem tworzenia produktu, decyduje o kolejności wykonywania operacji i udostępnia interfejs do konfiguracji budowniczego.

W powyższym przykładzie zdefiniowano klasę `Computer` reprezentującą skonfigurowany komputer. Klasa `ComputerBuilder` jest interfejsem dla klas budujących komputery, które implementują różne rodzaje komputerów, takie jak `GamingComputerBuilder` i `OfficeComputerBuilder`. Klasa `ComputerDirector` jest odpowiedzialna za konfigurację i zbudowanie komputera za pomocą odpowiedniego budowniczego.

W powyższym przykładzie, klasa `ComputerDirector` posiada pola komputera, takie jak CPU, RAM, storage i GPU, a także implementuje metody `buildCPU`, `buildRAM`, `buildStorage`, `buildGPU`, które pozwalają na skonfigurowanie komputera za pomocą odpowiedniego budowniczego.

W przykładzie użycia, tworzymy obiekt `ComputerDirector`, ustawiamy budowniczego na `GamingComputerBuilder` i budujemy komputer za pomocą metody `buildComputer()`. Następnie wykorzystując metodę `getComputer()` pobieramy zbudowany komputer.

Tak samo robimy z `OfficeComputerBuilder` i pobieramy zbudowany komputer.

W ten sposób, wzorzec Budowniczy pozwala na tworzenie skomplikowanych obiektów w sposób modularny i łatwy do rozszerzenia.

In [1]:
// https://www.digitalocean.com/community/tutorials/builder-design-pattern-in-java
public class Computer {

    //required parameters
    private String HDD;
    private String RAM;

    //optional parameters
    private boolean isGraphicsCardEnabled;
    private boolean isBluetoothEnabled;


    public String getHDD() {
        return HDD;
    }

    public String getRAM() {
        return RAM;
    }

    public boolean isGraphicsCardEnabled() {
        return isGraphicsCardEnabled;
    }

    public boolean isBluetoothEnabled() {
        return isBluetoothEnabled;
    }

    private Computer(ComputerBuilder builder) {
        this.HDD=builder.HDD;
        this.RAM=builder.RAM;
        this.isGraphicsCardEnabled=builder.isGraphicsCardEnabled;
        this.isBluetoothEnabled=builder.isBluetoothEnabled;
    }
    
    //Builder Class
    public static class ComputerBuilder{

        // required parameters
        private String HDD;
        private String RAM;

        // optional parameters
        private boolean isGraphicsCardEnabled;
        private boolean isBluetoothEnabled;

        public ComputerBuilder(String hdd, String ram){
            this.HDD=hdd;
            this.RAM=ram;
        }

        public ComputerBuilder setGraphicsCardEnabled(boolean isGraphicsCardEnabled) {
            this.isGraphicsCardEnabled = isGraphicsCardEnabled;
            return this;
        }

        public ComputerBuilder setBluetoothEnabled(boolean isBluetoothEnabled) {
            this.isBluetoothEnabled = isBluetoothEnabled;
            return this;
        }

        public Computer build(){
            return new Computer(this);
        }
    }
}

In [4]:
Computer comp = new Computer.ComputerBuilder("500 GB", "2 GB")
    .setBluetoothEnabled(true)
    .setGraphicsCardEnabled(true)
    .build();

Przykład: powiadomienia (Android)

In [ ]:
// kotlin
val builder = NotificationCompat.Builder(this, channelID)
    .setSmallIcon(R.drawable.ic_baseline_notifications_24)
    .setContentTitle("Powiadomienie")
    .setContentText("Treść powiadomienia")
    .setStyle(
        NotificationCompat.BigTextStyle()
            .bigText("Dalszy tekst powiadomienia")
    )
    .setContentIntent(pendingIntent)
    .setPriority(NotificationCompat.PRIORITY_DEFAULT)

with(NotificationManagerCompat.from(this)) {
    notify(notificationId, builder.build())
}

In [ ]:
// Java
NotificationCompat.Builder builder = new NotificationCompat.Builder(this, channelID)
    .setSmallIcon(R.drawable.ic_baseline_notifications_24)
    .setContentTitle("Powiadomienie")
    .setContentText("Treść powiadomienia")
    .setStyle(
            new NotificationCompat.BigTextStyle()
                    .bigText("Dalszy tekst powiadomienia")
    )
    .setContentIntent(pendingIntent)
    .setPriority(NotificationCompat.PRIORITY_DEFAULT);

NotificationManagerCompat notificationCompat = NotificationManagerCompat.from(this);
notificationCompat.notify(notificationId, builder.build());

## Obserwator

![](https://refactoring.guru/images/patterns/content/observer/observer-comic-1-en-2x.png)

Obserwator jest wzorcem projektowym, który pozwala na subskrypcję obiektów do otrzymywania powiadomień o zmianach stanu innego obiektu, zwanego obiektem obserwowanym. Wzorzec ten pozwala na dynamiczne dodawanie lub usuwanie subskrypcji bez konieczności modyfikowania kodu obiektu obserwowanego.

Korzystając z wzorca Obserwatora, można uniknąć tworzenia silnych powiązań między obiektami, które mogą być trudne do utrzymania i rozszerzania. Wzorzec ten jest często stosowany w projektowaniu interfejsów użytkownika, systemach wykrywania zmian w bazie danych, a także w wielu innych sytuacjach, gdzie istnieje potrzeba reagowania na zmiany stanu obiektów.

![](https://refactoring.guru/images/patterns/diagrams/observer/solution1-en-2x.png)

Obserwator składa się z dwóch głównych składników:

- Obiekt obserwowany (`Publisher`) - reprezentuje obiekt, którego stan jest monitorowany przez inne obiekty. Zawiera listę obiektów subskrybujących i metody do dodawania lub usuwania subskrybentów oraz wysyłania powiadomień o zmianie stanu.
- Obiekt subskrybujący (`Subscriber`) - reprezentuje obiekt, który jest zainteresowany otrzymywaniem powiadomień o zmianie stanu obiektu obserwowanego. Zawiera metodę aktualizującą, która jest wywoływana przez obiekt obserwowany w celu przekazania powiadomienia.

![](https://refactoring.guru/images/patterns/diagrams/observer/solution2-en-2x.png)

In [12]:
interface Subscriber {
    fun update(temperature: Float, humidity: Float, pressure: Float)
}

interface Publisher {
    fun registerSubscriber(subscriber: Subscriber)
    fun removeSubscriber(subscriber: Subscriber)
    fun notifySubscribers()
}

In [13]:
class WeatherData : Publisher {
    private val subscribers: MutableList<Subscriber> = ArrayList()
    private var temperature: Float = 0f
    private var humidity: Float = 0f
    private var pressure: Float = 0f

    override fun registerSubscriber(subscriber: Subscriber) {
        subscribers.add(subscriber)
    }

    override fun removeSubscriber(subscriber: Subscriber) {
        val i = subscribers.indexOf(subscriber)
        if (i >= 0) {
            subscribers.removeAt(i)
        }
    }

    override fun notifySubscribers() {
        for (subscriber in subscribers) {
            subscriber.update(temperature, humidity, pressure)
        }
    }

    fun setMeasurements(temperature: Float, humidity: Float, pressure: Float) {
        this.temperature = temperature
        this.humidity = humidity
        this.pressure = pressure
        measurementsChanged()
    }

    private fun measurementsChanged() {
        notifySubscribers()
    }
}

In [14]:
class CurrentConditionsDisplay (private val weatherData: Publisher) : Subscriber {
    private var temperature: Float = 0f
    private var humidity: Float = 0f

    init {
        weatherData.registerSubscriber(this)
    }

    override fun update(temperature: Float, humidity: Float, pressure: Float) {
        this.temperature = temperature
        this.humidity = humidity
        display()
    }

    fun display() {
        println("Current conditions: $temperature F degrees and $humidity % humidity")
    }
}

In [15]:
val weatherData = WeatherData()
val currentDisplay = CurrentConditionsDisplay(weatherData)
weatherData.setMeasurements(80f, 65f, 30.4f)

Current conditions: 80.0 F degrees and 65.0 % humidity


In [16]:
weatherData.setMeasurements(800f, 65f, 30.4f)

Current conditions: 800.0 F degrees and 65.0 % humidity


W powyższym przykładzie, klasa `WeatherData` jest obiektem obserwowanym, który przechowuje listę subskrybentów (obserwatorów) i powiadamia ich o zmianie danych pogodowych. Klasa `CurrentConditionsDisplay` jest przykładem obiektu subskrybującego, który otrzymuje powiadomienie o zmianie danych pogodowych i aktualizuje swój stan zgodnie z nowymi danymi.

W przykładzie użycia, tworzymy obiekt `WeatherData` i obiekt `CurrentConditionsDisplay`, który subskrybuje obiekt `WeatherData`. Następnie ustawiamy nowe dane pogodowe za pomocą metody `setMeasurements()` w obiekcie `WeatherData`. Metoda ta powoduje również wysłanie powiadomień do wszystkich subskrybentów, w tym do obiektu `CurrentConditionsDisplay`, który aktualizuje swój stan i wyświetla aktualne warunki pogodowe.

W ten sposób, wzorzec **Obserwator** pozwala na **dynamiczne** dodawanie lub usuwanie subskrypcji bez konieczności modyfikowania kodu obiektu obserwowanego. Jest to przydatne w sytuacjach, gdy wiele różnych obiektów potrzebuje reagować na zmiany stanu innego obiektu.

W ten sposób, wzorzec **Obserwator** pozwala na rozdzielenie odpowiedzialności pomiędzy obiektami, a także na unikanie tworzenia silnych powiązań między obiektami, co jest szczególnie przydatne w sytuacjach, gdy struktura aplikacji jest skomplikowana i trudna do utrzymania.

Przykład: `LiveData`, `ViewModel`

In [ ]:
// viewModel
private val _score = MutableLiveData(0)
val score: LiveData<Int>
    get() = _score

private val _currentWordCount = MutableLiveData(0)
val currentWordCount: LiveData<Int>
    get() = _currentWordCount

// Fragment
viewModel.currentScrambledWord.observe(viewLifecycleOwner) { newWord ->
            binding.textViewUnscrambledWord.text = newWord}

viewModel.score.observe(viewLifecycleOwner) {score ->
    binding.score.text = score.toString()}

viewModel.currentWordCount.observe(viewLifecycleOwner) {wordCount ->
    binding.wordCount.text = getString(
        R.string.word_count, wordCount, MAX_NO_OF_WORDS)}